In [13]:
from langchain.memory import ConversationSummaryMemory
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryMemory(llm=llm)


def add_message(input, output):
    memory.save_context({"input": input}, {"output": output})


def get_history():
    return memory.load_memory_variables({})


add_message("Hi I'm John, I live in South Korea", "Wow that is so cool!")

In [14]:
add_message("South Korea is so pretty", "I wish I could go!!!")

In [15]:
get_history()

{'history': 'John introduces himself as living in South Korea, and the AI expresses admiration for that fact. The human mentions how pretty South Korea is, and the AI expresses a wish to visit.'}